# Counting Machine Washer Parts in an Image using an Image Processing Pipeline

In [ ]:
from os import path
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2hsv
from skimage.filters import gaussian, threshold_multiotsu
from scipy import signal

if 'google.colab' in str(get_ipython()):
  from google.colab import files
  from google.colab.patches import cv2_imshow
  
  from google.colab import drive 
  drive.mount('/content/drive')

## Load Dataset

Run the following cell if images are to be uploaded

In [ ]:
#uploaded = files.upload()

Run the following cell if the dataset needs to be unzipped

In [ ]:
#if path.exists('/content/drive/Shareddrives/CIS680 - Final Project Fall\'21/dataset') == False:
   #os.mkdir('/content/drive/Shareddrives/CIS680 - Final Project Fall \'21/dataset')
#!unzip -u "/content/drive/Shareddrives/CIS680 - Final Project Fall '21/dataset.zip" -d "/content/drive/Shareddrives/CIS680 - Final Project Fall '21/dataset"

Run the following cell to load an individual image

In [ ]:
#src = cv2.imread("/content/0_angle0_img.png", cv2.IMREAD_COLOR)

## 1. Convert image to HSV

To isolate the Saturation channel

In [ ]:
def image_to_HSV(src, show = False , select_source = 2):
  '''
  Input:
    src:            Source Image
    show:           If set to True, displays converted image
    select_source:  1 for Hue channel, 2 for Saturation channel, 3 for Value channel
  
  Returns:
    source:         Selected channel of Image
  '''
    src_hsv = cv2.cvtColor(src, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(src_hsv)

    # Plot different Channels of HSV
    if show == True:
        fig, (ax0, ax1, ax2, ax3) = plt.subplots(ncols=4, figsize=(8, 2), dpi=190)
        ax0.imshow(cv2.cvtColor(src, cv2.COLOR_BGR2RGB))
        ax0.set_title("RGB image")
        ax0.axis('off')
        ax1.imshow(h, cmap='hsv')
        ax1.set_title("Hue channel")
        ax1.axis('off')
        ax2.imshow(s)
        ax2.set_title("Saturation channel")
        ax2.axis('off')
        ax3.imshow(v)
        ax3.set_title("Value channel")
        ax3.axis('off')
        fig.tight_layout()
    if select_source==1:
        source = h
    elif select_source==2:
        source = s
    elif select_source==3:
        source = v      
    # cv2_imshow(s)
    return source

## 2. Gaussian Filter

To remove noise from the image

In [ ]:
def Gaussian(source, show = False):
  '''
  Input:
    source:         Source image
    show:           If True, display Gaussian filtered image
  
  Returns:
    filtered_image: Gaussian filtered image
  '''
    filtered_image = cv2.GaussianBlur(source, (3,3),0,0)
    if show == True:
        fig, ax = plt.subplots(dpi=190)
        ax.imshow(filtered_image)
        ax.set_title("Gaussian Filtered Image")
        ax.axis('off')
        fig.tight_layout()
    # cv2_imshow(filtered_image)
    return filtered_image

## 3. Otsu Thresholding

Perform 2-level Otsu thresholding

In [ ]:
def Otsu(source, show = False):
  '''
  Input:
    source:             Source image
    show:               If True, display Otsu thresholded image
  
  Returns:
    thresholded_image:  Otsu thresholded image
  '''
    ret2, thresholded_image = cv2.threshold(source, 0,250, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    if show == True:
        fig, ax = plt.subplots(dpi=190)
        ax.imshow(thresholded_image)
        ax.set_title("Otsu Thresholded Image")
        ax.axis('off')
        fig.tight_layout()    
    # cv2_imshow(thresholded_image)
    return thresholded_image  

## 4. Sobel Edge Detection

Generate an edge map for the input image

In [ ]:
def Sobel(source, show = False):
  '''
  Input:
    source:             Source image
    show:               If True, display Sobel filtered image
  
  Returns:
    sobel_out:  Edge map of source image
  ''' 
    sobel_8u = cv2.Sobel(source, cv2.CV_64F, 0, 1, ksize=1)
    sobel_out = np.uint8(np.absolute(sobel_8u))
    if show == True:
        fig, ax = plt.subplots(dpi=190)
        ax.imshow(sobel_out)
        ax.set_title("Sobel Edge Detection")
        ax.axis('off')
        fig.tight_layout()
    # cv2_imshow(sobel_out)
    return sobel_out

## 5. Hough Transform

Applies a Hough Circle Transform to the generated edge map of the source image

NOTE: HoughCircles only takes in CV_8U format images

Inputting MaxRadius Values of <0 returns the centers without finding the respective radii

In [ ]:
def Circles(source, param1 = 100, param2 = 7, minRadius = 15, maxRadius = 25, show = False):
  '''
  Input:
    source:         Source image
    param1, param2: Parameters for Hough Circles
    minRadius:      Minimum radius of generated circles
    maxRadius:      Maximum radius of generated circles
    show:           If True, display image with generated circles
  
  Returns:
    Number of detected circles
  '''
#To detect outer circles: minRadius=15, maxRadius = 25, param2 = 6
#To detect inner circles: minRadius=5, maxRadius = 15, param2 = 6

    circles = cv2.HoughCircles(source, 
                              cv2.HOUGH_GRADIENT,
                              1,
                              20,
                              param1,
                              param2,
                              minRadius,
                              maxRadius)
    
    circles = np.uint16(np.around(circles[0]))
    #print(circles)
    if show == True:
        output = source.copy()
        # loop over the (x, y) coordinates and radius of the circles
        i = 1
        for (x, y, r) in circles:
          # draw the circle in the output image, then draw a rectangle
          # corresponding to the center of the circle
          cv2.circle(output, (x, y), r, (0, 255, 0), 4)
        # show the output image
        cv2_imshow(np.hstack([src, output]))
        cv2.waitKey(0)
    return len(circles)    

# Full Pipeline

## Obtaining Ground Truth Annotations 


Parsing the JSON file to get object count for each image

In [ ]:
#Function to get number of objects in the image
'''
def get_label(_id):
    annos = [s for s in imgs_anns["annotations"] if s["image_id"] == _id]
    return len(annos)
json_file = os.path.join("dataset", "annotations.json")
with open(json_file) as f:
  imgs_anns = json.load(f)
''' 

'\ndef get_label(_id):\n    annos = [s for s in imgs_anns["annotations"] if s["image_id"] == _id]\n    return len(annos)\njson_file = os.path.join("dataset", "annotations.json")\nwith open(json_file) as f:\n  imgs_anns = json.load(f)\n'

## Main function to run the whole pipeline

In [ ]:
def get_count(img, show = False):
  '''
  Inputs:
    img:    Source Image
    show:   If True, display processed images during intermediary steps

  Returns:
    count:  Count of circles detected in image
  '''
    img = image_to_HSV(img, show)
    img = Gaussian(img, show)
    img = Otsu(img, show)
    #cv2.imwrite("/content/drive/Shareddrives/CIS680 - Final Project Fall '21/OtsuBinaryImages/402_angle5_img.png", img)
    img = Sobel(img, show)
    count = Circles(img, show)
    return count

## Evaluation Metrics

Algorithm results evaluated on Root Mean Squared Error (RMSE) & Mean Absolute Error (MAE)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

pred_counts = []
gt_counts = []

for i in range (len(paths)):
    img = cv2.imread(image_name, cv2.BGR2RGB)
    pred_counts.append(getCount(img))
    gt_count.append(get_label(img_id))

pred_counts = np.array(pred_counts)
gt_counts = np.array(gt_counts)

rmse = sqrt(mean_squared_error(pred_counts, gt_counts))
mae = mean_absolute_error(pred_counts, gt_counts)

print('\n Root Mean Squared Error on Data: {}'.format(rmse))
print('\n Mean Absolute Error on Data: {}'.format(mae))